# 🦅 AVCI - Yüksek Oran Avcısı Eğitim Paneli v6.0.4 (Absolute Imports)
**Güncelleme Notları:**
- **Kesin Çözüm:** Tüm kodlar 'Relative Import' (.config) yerine 'Absolute Import' (config) yapısına geçirildi.
- Modül yükleme mantığı basitleştirildi.
---

In [ ]:
# 1. GitHub Deposunu Çek ve Kurulum Yap (Temiz Kurulum)
import os
import shutil

# 1. Kök dizine dön
try:
    os.chdir('/content')
except:
    pass

# 2. Varsa eski projeyi SİL (Her şeyi sıfırla ki eski hatalar kalmasın)
if os.path.exists('newproje'):
    print("Eski proje dosyaları temizleniyor...")
    shutil.rmtree('newproje')

# 3. Sıfırdan İndir
print("Projeyi GitHub'dan sıfırdan çekiyorum...")
!git clone https://github.com/onndd/newproje.git

# 4. Çalışma dizinine gir
if os.path.exists('newproje/avci'):
    os.chdir('newproje/avci')
    print(f"✅ Kurulum Başarılı. Çalışma Dizini: {os.getcwd()}")
else:
    print("❌ HATA: İndirme başarısız oldu!")

print("GPU Durumu:")
!nvidia-smi

# Kütüphane Kurulumları
!pip install optuna streamlit matplotlib pandas tensorflow hmmlearn plotly lightgbm
print("Kurulum ve Hazırlık Tamamlandı! Artık modüller bulunabilir.")

In [ ]:
# 2. Veri Yükleme ve Ayarlar
import optuna
import logging
import sys
import os

# Mevcut dizini yola ekle (Garanti olsun)
curr_dir = os.getcwd()
if curr_dir not in sys.path:
    sys.path.append(curr_dir)

optuna.logging.set_verbosity(optuna.logging.INFO)
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))

try:
    # Doğrudan import et (Reload kullanmıyoruz çünkü temiz kurulum yaptık)
    from train_avci import load_and_prep, train_target, visualize_performance
    from model_ae_avci import train_autoencoder
    from model_hmm_avci import train_hmm
    from optuna.visualization import plot_optimization_history, plot_param_importances
    print("✅ Modüller başarıyla yüklendi!")
except ImportError as e:
    print(f"❌ HATA: Modül import hatası. Detay: {e}")

if 'load_and_prep' in locals():
    df = load_and_prep(limit=100000)
    print(f"Veri Hazır: {len(df)} satır.")

## 🕵️‍♂️ Bölüm 1: İstihbarat & Komuta

In [ ]:
if 'train_autoencoder' in locals():
    ae = train_autoencoder(df)

In [ ]:
if 'train_hmm' in locals():
    hmm_model = train_hmm(df)

## 🎯 Bölüm 2: Nişancı Eğitimi (LightGBM + GPU)

In [ ]:
target = 1.5
model, X_val, y_val, features, study = train_target(df, target, epochs=30)
visualize_performance(model, X_val, y_val, target)
try: plot_optimization_history(study).show()
except: pass

In [ ]:
target = 3.0
model, X_val, y_val, features, study = train_target(df, target, epochs=30)
visualize_performance(model, X_val, y_val, target)
try: plot_optimization_history(study).show()
except: pass

In [ ]:
target = 5.0
model, X_val, y_val, features, study = train_target(df, target, epochs=50)
visualize_performance(model, X_val, y_val, target)
try: plot_optimization_history(study).show()
except: pass

In [ ]:
target = 10.0
model, X_val, y_val, features, study = train_target(df, target, epochs=50)
visualize_performance(model, X_val, y_val, target)
try: plot_optimization_history(study).show()
except: pass

In [ ]:
target = 50.0
model, X_val, y_val, features, study = train_target(df, target, epochs=100)
visualize_performance(model, X_val, y_val, target)
try: plot_optimization_history(study).show()
except: pass

In [ ]:
target = 100.0
model, X_val, y_val, features, study = train_target(df, target, epochs=100)
visualize_performance(model, X_val, y_val, target)
try: plot_optimization_history(study).show()
except: pass

In [ ]:
# GOD MODE
target = 1000.0
model, X_val, y_val, features, study = train_target(df, target, epochs=150)
visualize_performance(model, X_val, y_val, target)
try: plot_optimization_history(study).show()
except: pass

## 💾 Bölüm 3: Modelleri Kaydet ve İndir

In [ ]:
# Drive'ı Bağla (Opsiyonel - Yedekleme İçin)
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    print("Drive bağlanmadı veya erişim yok. Lokal indirme yapabilirsin.")

# Tüm Modelleri Ziple
!zip -r avci_models.zip models/

# İndir
try:
    from google.colab import files
    files.download('avci_models.zip')
    print("Dosya indiriliyor... 'avci_models.zip'")
except Exception as e:
    print(f"İndirme hatası: {e}. Sol taraftaki dosya gezgininden 'avci_models.zip' dosyasını sağ tıklayıp indirebilirsin.")

# Varsa Drive'a Kopyala
if os.path.exists('/content/drive/MyDrive'):
    !cp avci_models.zip /content/drive/MyDrive/avci_backup_latest.zip
    print("Yedek Drive'a (Ana Dizin) kopyalandı.")